In [1]:
import cv2
import random
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import os

In [2]:


def shuffle_image(img,windowSize):
	rows,cols,depth = img.shape
	piece_rows = rows // windowSize
	piece_cols = cols // windowSize
	rows -= rows % windowSize
	cols -= cols % windowSize
	new_image = np.zeros((rows,cols,depth))

	
	pieces = []

	for i in range(0,rows,piece_rows):
		for j in range(0,cols,piece_cols):
			pieces.append(img[i:i + piece_rows,j:j + piece_cols].copy())
	
	pieces_ind = [i for i in range(len(pieces))]
	random.shuffle(pieces_ind)
	shuffled_pieces = [pieces[i] for i in pieces_ind]


	# index = 0

	# for i in range(0,rows,piece_rows):
	# 	for j in range(0,cols,piece_cols):
	# 		new_image[i:i + piece_rows,j:j + piece_cols] = shuffled_pieces[index]
	# 		index += 1

	# cv2.imwrite("jumbled.jpg",new_image)
	return shuffled_pieces, pieces_ind


In [ ]:
img = cv2.imread("lfw_5590/Aaron_Eckhart_0001.jpg")
windowSize=2
shuffle_image(img,windowSize)

In [7]:
class FaceDataset(Dataset):
	def __init__(self,img_dir,windowSize,transform=None):
		self.img_dir = img_dir
		self.img_list = os.listdir(img_dir)
		self.transform = transform
		self.windowSize = windowSize

	def __len__(self):
		return len(self.img_list)

	def __getitem__(self,index):
		img = cv2.imread(self.img_dir + "/" + self.img_list[index])
		pieces, order = shuffle_image(img,self.windowSize)
		order = torch.tensor([int(x) for x in order])
		if self.transform:
			pieces = [self.transform(piece) for piece in pieces]

		return pieces, order
			

In [8]:
batch_size = 32

dataset = FaceDataset(img_dir="lfw_5590",windowSize=2,transform=transforms.ToTensor())
train_set, test_set = torch.utils.data.random_split(dataset,[4400,1190])
train_loader = DataLoader(dataset=train_set,batch_size=batch_size,shuffle=True)
test_loader =  DataLoader(dataset=test_set,batch_size=batch_size,shuffle=True)

In [9]:
train_features, train_labels = next(iter(train_loader))